In [1]:
# This notebook is an implementatio of an optical CNOT operation using PBSs. Based on: https://doi.org/10.1103/PhysRevA.64.062311
import qutip as qt
from swap_TN_direct import *

import qutip as qt
import numpy as np
from quimb.tensor.tensor_arbgeom import tensor_network_apply_op_vec

/home/asingal/home/.venv/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/asingal/home/.venv/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:
def quantum_encoder(mean_photon_num, N, psi_control, control_indices, error_tolerance):

    # Entangled state from EPS
    vacuum = create_vacuum_state(num_modes=8, N=N)
    bell_state, _ = light_source(vacuum, N, mean_photon_num, 8, error_tolerance, compress=True, contract=True)

    psi = extend_MPS(bell_state, psi_control)

    # PBS op: (The V mode is transmitted and not reflected)
    U_PBS_V = create_BS_MPO(site1 = 3, site2 = bell_state.L+control_indices[1], theta=np.pi/2, total_sites = 6, N = N, tag = r"$PBS$")
    enforce_1d_like(U_PBS_V, site_tags=psi.site_tags, inplace=True)
    psi = tensor_network_apply_op_vec(U_PBS_V, psi, compress=True, contract = True, cutoff = error_tolerance)

    # Measuring D_d
    # This is meant to change the basis from HV -> FS: (See https://doi.org/10.1103/PhysRevA.64.062311)
    U_PBS_FS = create_BS_MPO(site1 = 2, site2 = 3, theta=np.pi/4, total_sites = 6, N = N, tag = r"$rotator$")
    enforce_1d_like(U_PBS_FS, site_tags=psi.site_tags, inplace=True)
    psi = tensor_network_apply_op_vec(U_PBS_FS, psi, compress=True, contract = True, cutoff = error_tolerance)

    # Performing measurements:
    BSM_POVM_1_OPs = generate_sqrt_POVM_MPO(sites=[2], outcome = 1, total_sites=6, efficiency=1, N=N, pnr = True)
    BSM_POVM_1_OPs.extend(generate_sqrt_POVM_MPO(sites=[3], outcome = 0, total_sites=6, efficiency=1, N=N, pnr = True))

    for POVM_OP in BSM_POVM_1_OPs:
        psi = tensor_network_apply_op_vec(POVM_OP, psi, compress=True, contract = True, cutoff = error_tolerance)

    return psi


def destructive_CNOT(control_MPS, control_sites, target_MPS, target_sites, N, error_tolerance):
    
    psi = extend_MPS(target_MPS, control_MPS)

    U_rotator_FS = create_BS_MPO(site1 = target_MPS.L+control_sites[0], site2 = target_MPS.L+control_sites[1], theta=np.pi/4, total_sites = psi.L, N = N, tag = r"$rotator$")
    enforce_1d_like(U_rotator_FS, site_tags=psi.site_tags, inplace=True)
    psi = tensor_network_apply_op_vec(U_rotator_FS, psi, compress=True, contract = True, cutoff = error_tolerance)

    U_rotator_FS = create_BS_MPO(site1 = target_sites[0], site2 = target_sites[1], theta=np.pi/4, total_sites = psi.L, N = N, tag = r"$rotator$")
    enforce_1d_like(U_rotator_FS, site_tags=psi.site_tags, inplace=True)
    psi = tensor_network_apply_op_vec(U_rotator_FS, psi, compress=True, contract = True, cutoff = error_tolerance)

    # Creating PBS ops:
    # SWAP = qt.qip.operations.swap().full()
    # U_PBS_F = create_MPO(1, 3, psi.L, SWAP, N, r"$PBS$")
    U_PBS_F = create_BS_MPO(site1 = 1, site2 = 3, theta=np.pi/2, total_sites = 8, N = N, tag = r"$PBS$")
    enforce_1d_like(U_PBS_F, site_tags=psi.site_tags, inplace=True)
    psi = tensor_network_apply_op_vec(U_PBS_F, psi, compress=True, contract = True, cutoff = error_tolerance)

    U_inverse_rotator_FS = create_BS_MPO(site1 = target_sites[0], site2 = target_sites[1], theta=-np.pi/4, total_sites = psi.L, N = N, tag = r"$rotator$")
    enforce_1d_like(U_inverse_rotator_FS, site_tags=psi.site_tags, inplace=True)
    psi = tensor_network_apply_op_vec(U_inverse_rotator_FS, psi, compress=True, contract = True, cutoff = error_tolerance)

    U_inverse_rotator_FS = create_BS_MPO(site1 = target_MPS.L+control_sites[0], site2 = target_MPS.L+control_sites[1], theta=-np.pi/4, total_sites = psi.L, N = N, tag = r"$rotator$")
    enforce_1d_like(U_inverse_rotator_FS, site_tags=psi.site_tags, inplace=True)
    psi = tensor_network_apply_op_vec(U_inverse_rotator_FS, psi, compress=True, contract = True, cutoff = error_tolerance)

    BSM_POVM_1_OPs = generate_sqrt_POVM_MPO(sites=[2], outcome = 1, total_sites=8, efficiency=1, N=N, pnr = True)
    BSM_POVM_1_OPs.extend(generate_sqrt_POVM_MPO(sites=[3], outcome = 0, total_sites=8, efficiency=1, N=N, pnr = True))

    for POVM_OP in BSM_POVM_1_OPs:
        psi = tensor_network_apply_op_vec(POVM_OP, psi, compress=True, contract = True, cutoff = error_tolerance)

    return psi

def CNOT(psi_control, psi_control_modes, psi_target, psi_target_modes, N, mean_photon_num, error_tolerance):

    psi = quantum_encoder(mean_photon_num, N, psi_control, psi_control_modes, error_tolerance)
    psi = destructive_CNOT(psi, (0,1), psi_target, psi_target_modes, N, error_tolerance)

    norm = psi.normalize()
    for _ in range(4):
        psi.measure(2, remove = True, renorm = True, inplace = True)
    psi[-1].modify(data=psi[-1].data * norm**0.5)

    return psi


In [8]:
N = 2
mean_photon_num = 0.05
error_tolerance = 1e-10

In [9]:
# Defining the control and target inputs here
a_dag = qt.create(N).full()
I = qt.identity(N).full()

a_H = np.kron(a_dag, I)
a_V = np.kron(I, a_dag)

vacuum = create_vacuum_state(num_modes=2, N=N)

control_operator = 1/np.sqrt(3) * a_H + np.sqrt(2/3) * a_V
control_MPO = create_MPO(0, 1, 2, control_operator, N, "control")
psi_control = tensor_network_apply_op_vec(control_MPO, vacuum, compress=True, contract = True, cutoff = error_tolerance)

# Our target is in the H state 
target_operator = a_V
target_MPO = create_MPO(0, 1, 2, target_operator, N, "target")
psi_target = tensor_network_apply_op_vec(target_MPO, vacuum, compress=True, contract = True, cutoff = error_tolerance)

In [10]:
psi = CNOT(psi_control, (0,1), psi_target, (0,1), N, mean_photon_num, error_tolerance)

read_quantum_state(psi, N, num_states=2)

Corresponding Basis terms:
0H0V_B 0H0V_A - 0 - [0.28795405-0.j]
0H1V_B 1H0V_A - 6 - [0.+0.01018921j]
1H0V_B 0H1V_A - 9 - [0.+0.01440971j]
1H1V_B 1H1V_A - 15 - [0.00050989+0.j]
